In [1]:
import pandas as pd
from pathlib import Path
import os
import zipfile
import calendar
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
fileName = 'data_for_AsteriosIlektra.zip'
filePath = 'data_for_AsteriosIlektra'
data_dir = Path(filePath)

In [3]:
if not os.path.exists(filePath):
    print('Data have not been extracted.')
    with zipfile.ZipFile(fileName, 'r') as zip_ref:
        zip_ref.extractall()
else:
    print('Data have already been extracted.')

Data have already been extracted.


In [4]:
if not os.path.exists('data/data.csv'):
    df = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')
    )

    df.to_csv('data/data.csv')
else:
    df = pd.read_csv('data/data.csv').drop(columns=['Unnamed: 0'])

**Taking a smaller sample (maybe in the future we can change how we take our sample data)**

In [5]:
sub_df = df.iloc[:10000]
sub_df

,start_time_unix,end_time_unix,user_id
0,1482559891,1482559981,8953
1,1486570973,1486570993,3633
2,1481305012,1481305022,3633
3,1528060816,1528060826,3633
4,1474017399,1474017409,3633
...,...,...,...
9995,1529338887,1529338897,8774
9996,1517861512,1517861585,8774
9997,1526452547,1526452678,906
9998,1480942312,1480942413,906


## Data Preprocessing

In [6]:
sub_df[['start_time', 'end_time']] = sub_df[['start_time_unix', 'end_time_unix']].apply(pd.to_datetime, unit='s', origin='unix')
sub_df['duration'] = sub_df['end_time'] - sub_df['start_time']

In [7]:
def hour_rounder(t):
    if t.minute < 30:
        return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour))
    else:
        return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(minutes=30))

In [8]:
def hour_reference(t): #change this to remove the year infront
    if t.minute < 30:
        return (str(t.hour).zfill(2)+':00'+ '-' + str(t.hour).zfill(2) + ':30')
#    elif t.hour == 23 & t.minute >= 30:
#        return (str(t.hour).zfill(2)+':'+'30' + '-' + '00:00')
    else:
        return (str(t.hour).zfill(2)+':'+'30' + '-' + str(t.hour + 1).zfill(2)+':00')

In [9]:
# hour_reference(sub_df.iloc[38])

In [10]:
sub_df['range_start_time'] = sub_df['start_time'].apply(lambda x: hour_rounder(x))
sub_df['range_end_time'] = sub_df['range_start_time'].apply(lambda x: x+timedelta(minutes=30))

In [11]:
sub_df['hour_period'] = sub_df['range_start_time'].apply(lambda x: hour_reference(x))
#sub_df['hour_end_mark'] = sub_df['range_end_time'].apply(lambda x: hour_reference(x))

In [12]:
sub_df['year'] = pd.to_datetime(sub_df['start_time']).dt.year
sub_df['month_numeric'] = pd.to_datetime(sub_df['start_time']).dt.month
sub_df['weekday'] = pd.to_datetime(sub_df['start_time']).dt.weekday # monday is 0
sub_df['day_numeric'] = pd.to_datetime(sub_df['start_time']).dt.day
sub_df['Day'] = pd.to_datetime(sub_df['start_time']).dt.strftime('%A') #day numeric
sub_df['hour'] = pd.to_datetime(sub_df['start_time']).dt.hour
sub_df['minute'] = pd.to_datetime(sub_df['start_time']).dt.minute
sub_df['second'] = pd.to_datetime(sub_df['start_time']).dt.second
sub_df['date'] = pd.to_datetime(sub_df['start_time']).dt.date

In [13]:
cats = [
    '00:00-00:30', '00:30-01:00', '01:00-01:30', '01:30-02:00', '02:00-02:30', '02:30-03:00', '03:00-03:30', 
    '03:30-04:00', '04:00-04:30', '04:30-05:00','05:00-05:30', '05:30-06:00', '06:00-06:30', '06:30-07:00', '07:00-07:30', 
    '07:30-08:00','08:00-08:30','08:30-09:00', '09:00-09:30', '09:30-10:00', '10:00-10:30', '10:30-11:00','11:00-11:30',
    '11:30-12:00', '12:00-12:30', '12:30-13:00', '13:00-13:30', '13:30-14:00','14:00-14:30',
    '14:30-15:00', '15:00-15:30', '15:30-16:00', '16:00-16:30', '16:30-17:00', '17:00-17:30', '17:30-18:00', '18:00-18:30', 
    '18:30-19:00', '19:00-19:30', '19:30-20:00','20:00-20:30', '20:30-21:00', '21:00-21:30', '21:30-22:00', '22:00-22:30', 
    '22:30-23:00','23:00-23:30', '23:30-24:00'
    ]
sub_df['hour_period'] = pd.Categorical(sub_df['hour_period'], categories=cats, ordered=True)
#sub_df['hour_end_mark'] = pd.Categorical(sub_df['hour_end_mark'], categories=cats, ordered=True)

# to right order day strings
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sub_df['day'] = pd.Categorical(sub_df['Day'], categories=cats, ordered=True)

# to change month into month strings
sub_df['month'] = sub_df['month_numeric'].apply(lambda x: calendar.month_abbr[x])
cats = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',  'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec',]
sub_df['month'] = pd.Categorical(sub_df['month'], categories=cats, ordered=True)

In [14]:
def timedelta_to_minutes(td):
    return round(td.seconds/60, 3)

sub_df['duration_min'] = sub_df['duration'].apply(lambda x: timedelta_to_minutes(x))

In [15]:
sub_df.iloc[4184]

start_time_unix              1518220371
end_time_unix                1518257207
user_id                            6277
start_time          2018-02-09 23:52:51
end_time            2018-02-10 10:06:47
duration                0 days 10:13:56
range_start_time    2018-02-09 23:30:00
range_end_time      2018-02-10 00:00:00
hour_period                 23:30-24:00
year                               2018
month_numeric                         2
weekday                               4
day_numeric                           9
Day                              Friday
hour                                 23
minute                               52
second                               51
date                         2018-02-09
day                              Friday
month                               Feb
duration_min                    613.933
Name: 4184, dtype: object

# Picking the sample dataset

In [16]:
sub_df

,start_time_unix,end_time_unix,user_id,start_time,end_time,duration,range_start_time,range_end_time,hour_period,year,...,weekday,day_numeric,Day,hour,minute,second,date,day,month,duration_min
0,1482559891,1482559981,8953,2016-12-24 06:11:31,2016-12-24 06:13:01,00:01:30,2016-12-24 06:00:00,2016-12-24 06:30:00,06:00-06:30,2016,...,5,24,Saturday,6,11,31,2016-12-24,Saturday,Dec,1.500
1,1486570973,1486570993,3633,2017-02-08 16:22:53,2017-02-08 16:23:13,00:00:20,2017-02-08 16:00:00,2017-02-08 16:30:00,16:00-16:30,2017,...,2,8,Wednesday,16,22,53,2017-02-08,Wednesday,Feb,0.333
2,1481305012,1481305022,3633,2016-12-09 17:36:52,2016-12-09 17:37:02,00:00:10,2016-12-09 17:30:00,2016-12-09 18:00:00,17:30-18:00,2016,...,4,9,Friday,17,36,52,2016-12-09,Friday,Dec,0.167
3,1528060816,1528060826,3633,2018-06-03 21:20:16,2018-06-03 21:20:26,00:00:10,2018-06-03 21:00:00,2018-06-03 21:30:00,21:00-21:30,2018,...,6,3,Sunday,21,20,16,2018-06-03,Sunday,Jun,0.167
4,1474017399,1474017409,3633,2016-09-16 09:16:39,2016-09-16 09:16:49,00:00:10,2016-09-16 09:00:00,2016-09-16 09:30:00,09:00-09:30,2016,...,4,16,Friday,9,16,39,2016-09-16,Friday,Sep,0.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1529338887,1529338897,8774,2018-06-18 16:21:27,2018-06-18 16:21:37,00:00:10,2018-06-18 16:00:00,2018-06-18 16:30:00,16:00-16:30,2018,...,0,18,Monday,16,21,27,2018-06-18,Monday,Jun,0.167
9996,1517861512,1517861585,8774,2018-02-05 20:11:52,2018-02-05 20:13:05,00:01:13,2018-02-05 20:00:00,2018-02-05 20:30:00,20:00-20:30,2018,...,0,5,Monday,20,11,52,2018-02-05,Monday,Feb,1.217
9997,1526452547,1526452678,906,2018-05-16 06:35:47,2018-05-16 06:37:58,00:02:11,2018-05-16 06:30:00,2018-05-16 07:00:00,06:30-07:00,2018,...,2,16,Wednesday,6,35,47,2018-05-16,Wednesday,May,2.183
9998,1480942312,1480942413,906,2016-12-05 12:51:52,2016-12-05 12:53:33,00:01:41,2016-12-05 12:30:00,2016-12-05 13:00:00,12:30-13:00,2016,...,0,5,Monday,12,51,52,2016-12-05,Monday,Dec,1.683


In [17]:
sample_df = sub_df[['user_id', 'date', 'year', 'month', 'day', 'hour', 'minute', 'second', 'hour_period', 'duration_min', 'start_time',	'end_time']]
sample_df

,user_id,date,year,month,day,hour,minute,second,hour_period,duration_min,start_time,end_time
0,8953,2016-12-24,2016,Dec,Saturday,6,11,31,06:00-06:30,1.500,2016-12-24 06:11:31,2016-12-24 06:13:01
1,3633,2017-02-08,2017,Feb,Wednesday,16,22,53,16:00-16:30,0.333,2017-02-08 16:22:53,2017-02-08 16:23:13
2,3633,2016-12-09,2016,Dec,Friday,17,36,52,17:30-18:00,0.167,2016-12-09 17:36:52,2016-12-09 17:37:02
3,3633,2018-06-03,2018,Jun,Sunday,21,20,16,21:00-21:30,0.167,2018-06-03 21:20:16,2018-06-03 21:20:26
4,3633,2016-09-16,2016,Sep,Friday,9,16,39,09:00-09:30,0.167,2016-09-16 09:16:39,2016-09-16 09:16:49
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8774,2018-06-18,2018,Jun,Monday,16,21,27,16:00-16:30,0.167,2018-06-18 16:21:27,2018-06-18 16:21:37
9996,8774,2018-02-05,2018,Feb,Monday,20,11,52,20:00-20:30,1.217,2018-02-05 20:11:52,2018-02-05 20:13:05
9997,906,2018-05-16,2018,May,Wednesday,6,35,47,06:30-07:00,2.183,2018-05-16 06:35:47,2018-05-16 06:37:58
9998,906,2016-12-05,2016,Dec,Monday,12,51,52,12:30-13:00,1.683,2016-12-05 12:51:52,2016-12-05 12:53:33


# Features 

## Jumps

In [18]:
jumps = sub_df.groupby(['hour_period', 'user_id'])['user_id'].count().reset_index(name ='Num Session')
jumps = jumps.pivot_table('Num Session', ['hour_period'], 'user_id')
jumps = jumps.fillna(0)

In [19]:
jumps.T

hour_period,00:00-00:30,00:30-01:00,01:00-01:30,01:30-02:00,02:00-02:30,02:30-03:00,03:00-03:30,03:30-04:00,04:00-04:30,04:30-05:00,...,19:00-19:30,19:30-20:00,20:00-20:30,20:30-21:00,21:00-21:30,21:30-22:00,22:00-22:30,22:30-23:00,23:00-23:30,23:30-24:00
user_id,,,,,,,,,,,,,,,,,,,,,
389,8,9,3,2,2,3,10,12,13,18,...,66,77,53,50,64,44,32,25,16,7
906,0,0,0,0,0,0,0,0,1,7,...,22,16,17,19,10,13,7,4,3,2
1491,0,0,0,0,0,0,0,1,0,1,...,7,5,0,1,0,0,0,1,0,0
1559,1,0,0,0,1,0,0,0,5,12,...,25,23,32,18,26,17,10,9,1,4
1562,0,0,0,0,1,2,9,23,30,36,...,8,1,4,2,1,1,0,0,0,0
2437,5,3,2,2,3,2,1,5,7,3,...,53,43,44,28,40,28,25,16,15,3
3633,5,4,3,4,0,5,2,1,2,4,...,21,15,18,16,11,15,8,8,5,6
5773,1,0,0,0,0,0,0,2,2,11,...,30,23,8,5,4,0,1,1,0,0
6277,6,3,3,3,4,0,3,0,4,0,...,16,29,21,25,20,23,11,12,8,5


## Duration of session per 30 mins

In [20]:
activity2 = sub_df.groupby(['hour_period', 'user_id'])['duration_min'].sum().reset_index(name ='Num Session')
activity2 = activity2.pivot_table('Num Session', ['hour_period'], 'user_id')
activity2 = activity2.fillna(0)
activity2.T

hour_period,00:00-00:30,00:30-01:00,01:00-01:30,01:30-02:00,02:00-02:30,02:30-03:00,03:00-03:30,03:30-04:00,04:00-04:30,04:30-05:00,...,19:00-19:30,19:30-20:00,20:00-20:30,20:30-21:00,21:00-21:30,21:30-22:00,22:00-22:30,22:30-23:00,23:00-23:30,23:30-24:00
user_id,,,,,,,,,,,,,,,,,,,,,
389,8.667,21.282,2.433,1.000,0.967,3.716,5.984,5.034,16.833,11.434,...,81.504,76.789,79.288,49.169,80.105,103.069,71.488,47.215,37.484,12.367
906,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.333,31.784,...,81.999,29.201,52.419,44.101,56.067,14.684,9.966,6.967,2.667,7.000
1491,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,1.167,...,10.451,2.084,0.000,0.117,0.000,0.000,0.000,1.833,0.000,0.000
1559,1.133,0.000,0.000,0.000,0.167,0.000,0.000,0.000,8.733,18.484,...,97.518,77.452,94.084,60.717,72.333,87.150,5.933,22.900,8.333,3.334
1562,0.000,0.000,0.000,0.000,15.017,2.333,60.052,129.016,111.020,196.422,...,21.017,0.133,13.183,1.000,0.167,0.667,0.000,0.000,0.000,0.000
2437,4.000,4.184,0.766,2.667,8.017,25.634,0.100,3.500,8.318,3.201,...,63.552,86.871,66.287,33.851,42.121,57.900,33.367,30.368,26.303,6.850
3633,11.767,6.800,42.900,6.334,0.000,7.267,1.850,0.167,0.500,19.699,...,75.401,20.785,44.435,77.517,43.652,45.248,10.001,12.183,10.716,23.933
5773,0.167,0.000,0.000,0.000,0.000,0.000,0.000,0.516,2.500,10.301,...,76.884,99.551,14.516,39.667,5.684,0.000,1.967,0.217,0.000,0.000
6277,481.016,56.567,8.117,1.500,7.267,0.000,6.734,0.000,15.150,0.000,...,26.001,78.484,77.252,101.734,197.616,70.519,24.451,292.401,49.901,627.517


In [21]:
activity = sub_df.groupby(['hour_period', 'user_id'])['duration_min'].sum().reset_index(name='Activity')
activity['Activity'] = activity['Activity'].fillna(0)
activity = activity.set_index('hour_period')
activity

,user_id,Activity
hour_period,,
00:00-00:30,389,8.667
00:00-00:30,906,0.000
00:00-00:30,1491,0.000
00:00-00:30,1559,1.133
00:00-00:30,1562,0.000
...,...,...
23:30-24:00,6476,0.000
23:30-24:00,8774,1.250
23:30-24:00,8836,0.000


Activity for one user:

In [22]:
sub_df[sub_df['user_id'] == 906].groupby(['hour_period'])['duration_min'].sum()

hour_period
00:00-00:30      0.000
00:30-01:00      0.000
01:00-01:30      0.000
01:30-02:00      0.000
02:00-02:30      0.000
02:30-03:00      0.000
03:00-03:30      0.000
03:30-04:00      0.000
04:00-04:30      1.333
04:30-05:00     31.784
05:00-05:30     55.450
05:30-06:00     36.866
06:00-06:30     68.050
06:30-07:00     47.869
07:00-07:30    102.502
07:30-08:00     56.767
08:00-08:30     57.986
08:30-09:00     92.602
09:00-09:30     19.267
09:30-10:00     42.434
10:00-10:30     88.267
10:30-11:00     46.868
11:00-11:30     91.366
11:30-12:00     26.900
12:00-12:30     13.133
12:30-13:00     41.033
13:00-13:30     22.316
13:30-14:00     17.652
14:00-14:30     19.552
14:30-15:00     15.584
15:00-15:30    123.450
15:30-16:00     14.801
16:00-16:30     52.034
16:30-17:00     36.720
17:00-17:30     39.050
17:30-18:00     34.568
18:00-18:30     29.418
18:30-19:00     47.365
19:00-19:30     81.999
19:30-20:00     29.201
20:00-20:30     52.419
20:30-21:00     44.101
21:00-21:30     56.067

In [23]:
sample_df.to_csv('data/sample_data.csv')

In [24]:
sample_df[sample_df.hour_period.isna()]

,user_id,date,year,month,day,hour,minute,second,hour_period,duration_min,start_time,end_time


In [25]:
sample_df['session'] = 1

In [26]:
sample_df.to_csv('data/sample_data.csv')

# For a user / testing on Observable

In [27]:
sample_df

,user_id,date,year,month,day,hour,minute,second,hour_period,duration_min,start_time,end_time,session
0,8953,2016-12-24,2016,Dec,Saturday,6,11,31,06:00-06:30,1.500,2016-12-24 06:11:31,2016-12-24 06:13:01,1
1,3633,2017-02-08,2017,Feb,Wednesday,16,22,53,16:00-16:30,0.333,2017-02-08 16:22:53,2017-02-08 16:23:13,1
2,3633,2016-12-09,2016,Dec,Friday,17,36,52,17:30-18:00,0.167,2016-12-09 17:36:52,2016-12-09 17:37:02,1
3,3633,2018-06-03,2018,Jun,Sunday,21,20,16,21:00-21:30,0.167,2018-06-03 21:20:16,2018-06-03 21:20:26,1
4,3633,2016-09-16,2016,Sep,Friday,9,16,39,09:00-09:30,0.167,2016-09-16 09:16:39,2016-09-16 09:16:49,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8774,2018-06-18,2018,Jun,Monday,16,21,27,16:00-16:30,0.167,2018-06-18 16:21:27,2018-06-18 16:21:37,1
9996,8774,2018-02-05,2018,Feb,Monday,20,11,52,20:00-20:30,1.217,2018-02-05 20:11:52,2018-02-05 20:13:05,1
9997,906,2018-05-16,2018,May,Wednesday,6,35,47,06:30-07:00,2.183,2018-05-16 06:35:47,2018-05-16 06:37:58,1
9998,906,2016-12-05,2016,Dec,Monday,12,51,52,12:30-13:00,1.683,2016-12-05 12:51:52,2016-12-05 12:53:33,1


In [54]:
other_user_389_final = sample_df[sample_df["user_id"] == 389][['start_time', 'end_time', 'day', 'duration_min', 'session']]

In [29]:
other_user_389_final.to_csv('data/other_user_389_final.csv')

# for calendar;

In [177]:
calendar_user_389_final = sample_df[sample_df["user_id"] == 389].groupby("date")[['duration_min', 'session']].sum().reset_index()

calendar_user_389_final['day'] = pd.to_datetime(calendar_user_389_final['date']).dt.strftime('%A') #day numeric

calendar_user_389_final = calendar_user_389_final.set_index('date')

In [178]:
idx = pd.date_range(str(calendar_user_389_final.index[0]), str(calendar_user_389_final.index[-1]))

calendar_user_389_final.index = pd.DatetimeIndex(calendar_user_389_final.index)

calendar_user_389_final = calendar_user_389_final.reindex(idx, fill_value=0)

# calendar_user_389_final_reseted = calendar_user_389_final.reset_index()

# calendar_user_389_final['day'] = pd.to_datetime(calendar_user_389_final_reseted['index']).dt.strftime('%A') #day numeric
# print(s)
calendar_user_389_final

,duration_min,session,day
2017-06-03,0.333,1,Saturday
2017-06-04,2.868,5,Sunday
2017-06-05,1.334,4,Monday
2017-06-06,0.334,2,Tuesday
2017-06-07,11.250,3,Wednesday
...,...,...,...
2018-12-17,2.367,3,Monday
2018-12-18,16.800,7,Tuesday
2018-12-19,6.100,5,Wednesday
2018-12-20,4.733,4,Thursday


In [179]:
resetted = calendar_user_389_final.reset_index()

resetted['day'] = pd.to_datetime(resetted['index']).dt.strftime('%A') #day numeric

In [180]:
resetted[resetted['duration_min'] == 0]

,index,duration_min,session,day
6,2017-06-09,0.0,0,Friday
7,2017-06-10,0.0,0,Saturday
8,2017-06-11,0.0,0,Sunday
9,2017-06-12,0.0,0,Monday
10,2017-06-13,0.0,0,Tuesday
11,2017-06-14,0.0,0,Wednesday
12,2017-06-15,0.0,0,Thursday
13,2017-06-16,0.0,0,Friday
14,2017-06-17,0.0,0,Saturday
15,2017-06-18,0.0,0,Sunday


In [156]:
# calendar_user_389_final.set_index('date').duration_min.asfreq('duration_min')

In [157]:
# calendar_user_389_final.set_index('date')

In [158]:
# calendar_user_389_final.set_index('date').asfreq('date')

In [93]:
# calendar_user_389_final['day'] = pd.to_datetime(calendar_user_389_final['date']).dt.strftime('%A') #day numeric

In [181]:
resetted

,index,duration_min,session,day
0,2017-06-03,0.333,1,Saturday
1,2017-06-04,2.868,5,Sunday
2,2017-06-05,1.334,4,Monday
3,2017-06-06,0.334,2,Tuesday
4,2017-06-07,11.250,3,Wednesday
...,...,...,...,...
562,2018-12-17,2.367,3,Monday
563,2018-12-18,16.800,7,Tuesday
564,2018-12-19,6.100,5,Wednesday
565,2018-12-20,4.733,4,Thursday


In [97]:
# days = sample_df.merge(calendar_user_389_final, how='right')[['date', 'day']]
# days

In [96]:
# calendar_user_389_final.merge(days, how='right', on='date').head(10)

In [182]:
calendar_user_389_final_daily_average = sample_df[sample_df["user_id"] == 389].groupby("day")[['duration_min']].mean() 
calendar_user_389_final_daily_average = calendar_user_389_final_daily_average.rename({'duration_min' : 'Dialy_Average'}, axis='columns').round(1)

calendar_user_389_final_daily_average

# calendar_user_389_final.merge(calendar_user_389_final_daily_average, how='left', on='day')

# calendar_user_389_final_daily_average

,Dialy_Average
day,
Monday,1.4
Tuesday,1.4
Wednesday,1.4
Thursday,1.3
Friday,1.3
Saturday,1.6
Sunday,1.5


In [183]:
calendar_user_389_final_w_daily = resetted.merge(calendar_user_389_final_daily_average, how='left', on='day')

In [170]:
# f = {'duration_min':['sum','mean'], 'B':['prod'], 'D': {'my name': cust}}




# calendar_user_389_final_day = sample_df[sample_df["user_id"] == 389].groupby("day")[['duration_min', 'session']].sum() 

In [187]:
calendar_user_389_final_w_daily['percentage'] = (calendar_user_389_final_w_daily['duration_min'] - calendar_user_389_final_w_daily['Dialy_Average']) / calendar_user_389_final_w_daily['Dialy_Average']

In [188]:
calendar_user_389_final_w_daily[(calendar_user_389_final_w_daily['index'] >= '2018-01-01') & (calendar_user_389_final_w_daily['index'] <= '2018-01-31')]   

,index,duration_min,session,day,Dialy_Average,percentage
212,2018-01-01,9.583,5,Monday,1.4,5.845000
213,2018-01-02,2.184,4,Tuesday,1.4,0.560000
214,2018-01-03,1.167,1,Wednesday,1.4,-0.166429
215,2018-01-04,0.567,3,Thursday,1.3,-0.563846
216,2018-01-05,5.001,5,Friday,1.3,2.846923
217,2018-01-06,5.917,6,Saturday,1.6,2.698125
218,2018-01-07,0.500,2,Sunday,1.5,-0.666667
219,2018-01-08,4.067,5,Monday,1.4,1.905000
220,2018-01-09,5.799,5,Tuesday,1.4,3.142143
221,2018-01-10,5.668,4,Wednesday,1.4,3.048571


In [189]:
calendar_user_389_final_w_daily.to_csv('data/calendar_user_389_final_w_dailyduration.csv') #updated data for weekly average observable visualization

In [31]:
calendar_user_389_final.to_csv('data/calendar_user_389_final.csv')

PermissionError: [Errno 13] Permission denied: 'data/calendar_user_389_final.csv'

In [32]:
sample_df.groupby('date').mean()['duration_min'].min()

0.20850000000000002

In [33]:
sample_df.groupby('date').mean()['duration_min'].max()

71.19444444444444

# Average stats

In [34]:
import numpy as np

np.mean(sample_df.duration_min)

2.0949852

In [35]:
np.min(sample_df.duration_min)

0.017

In [36]:
 np.max(sample_df.duration_min)

613.933

In [37]:
(np.max(sample_df.duration_min) - np.min(sample_df.duration_min)) / np.max(sample_df.duration_min)

0.999972309682001